In [ ]:
import sys 
sys.path.append('../backtest')

In [ ]:
import populator

In [ ]:
data_dir = 'data'
exchanges = [('binance', 1000)]
shit_coins = ['ETH/BTC', 'LTC/BTC', 'TRX/BTC', 'ICX/BTC', 'OMG/BTC', 'NEO/BTC']
pairs = ['BTC/USDT'] + shit_coins
tick_size = '1m'
start = '2019-01-01T00:00:00Z'
num_ticks = 31 * 24 * 60

In [ ]:
populator.populate(data_dir, exchanges, pairs, tick_size, start, num_ticks)

In [ ]:
import pandas as pd
from functools import reduce

def load_price_frame(pair):
    df = populator.load_data_as_frame(data_dir, 'binance', pair, tick_size, start, num_ticks)
    df = df.rename(columns = {'close': pair})
    return df[[pair]]

def merge_price_frames(pairs):
    dfs = [load_price_frame(pair) for pair in pairs]
    return reduce(lambda left, right: pd.merge(left, right, on='timestamp'), dfs)

In [ ]:
prices = merge_price_frames(pairs)

In [ ]:
import matplotlib.pyplot as plt

def cross_corr(data_x, data_y, lag=0):
    return data_x.corr(data_y.shift(-lag))

def plot_lag(shit_coin, max_lag_offset):
    data = [abs(cross_corr(prices['BTC/USDT'], prices[shit_coin], lag=i)) for i in range(max_lag_offset)]
    plt.xlabel('Lag Time (minutes)')
    plt.ylabel('Abs. Correlation (to BTC/USDT)')
    plt.plot(data, label=shit_coin)
    plt.legend(loc='lower right')
    plt.show()

In [ ]:
plot_lag('ETH/BTC', 360)

In [ ]:
plot_lag('TRX/BTC', 360)

In [ ]:
plot_lag('OMG/BTC', 360)